In [2]:
#| default_exp 49-encoder-parallel-ngame-for-wikititles

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from xcai.basics import *
from xcai.models.PPP0XX import DBT009

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT']='xc-nlg_34-ngame-training-pipeline-with-multitriplet-loss-with-clustering-for-wikititles'

In [8]:
#| export
data_dir = '/home/aiscuser/scratch/datasets'

In [11]:
block = XCBlock.from_cfg(data_dir, 'data', dset='wikititles', valid_pct=0.001, tfm='xcnlg', 
                         tokenizer='distilbert-base-uncased', smp_features=[('lbl2data',1,2)])

In [12]:
#| export
pkl_file = f'{data_dir}/processed/wikititles_data_distilbert-base-uncased_xcnlg_ngame.pkl'

In [13]:
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [8]:
#| export
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [9]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/49-encoder-parallel-ngame-for-wikititles-1-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=10,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
#| export
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [11]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('distilbert-base-uncased', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, 
                               n_negatives=5, apply_softmax=True)
model.init_dr_head()

Some weights of DBT009 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.module.dr_layer_norm.bias', 'encoder.module.dr_layer_norm.weight', 'encoder.module.dr_projector.bias', 'encoder.module.dr_projector.weight', 'encoder.module.dr_transform.bias', 'encoder.module.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

In [30]:
learn.train()

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()